In [96]:
!pip install yfinance
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier


In [88]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","BIM","AI","AMZN"]
symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
dfs = []
for i in symbols:
   data = yf.download(i,period="MAX",interval="1mo")
   dfs.append(data)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [89]:
fixed_dfs = []
for df in dfs:
    full = []
    df = np.array(df)
    for i in range(15, df.shape[0]-1):
        fifteen_days_ago = i-15
        fourteen_days_ago = i-14
        thirteen_days_ago = i-13
        twelve_days_ago = i-12
        eleven_days_ago = i-11
        ten_days_ago = i-10
        nine_days_ago = i-9
        eight_days_ago = i-8
        seven_days_ago = i-7
        six_days_ago = i-6
        five_days_ago = i-5
        four_days_ago = i-4
        three_days_ago = i-3
        two_days_ago = i - 2
        yesterday = i - 1
        today = i
        tomorrow = i + 1
        if df[tomorrow][3] > df[today][3]:
            future = 1
        else:
            future = 0


        new_lst = [df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],df[twelve_days_ago][3],df[eleven_days_ago][3],
                   df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                   df[today][3], future]

        full.append(new_lst)


    fuu = np.array([full])
    lst = []
    for i in range(0, fuu.shape[1]):
        lst.append(fuu[0][i])
    
    mm = np.array(lst)


    column_names = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today", "result"]
    
    cnwr = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today"]
    dff = pd.DataFrame(mm, columns=column_names)
    
    scaler = MinMaxScaler()
    xx = dff["result"]
    dff = dff.drop(["result"],axis=1)
    scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=cnwr)

    scaled["result"] = xx
       
    fixed_dfs.append(scaled)

data = fixed_dfs[0]
for i in range(1,len(symbols)):
  data = pd.concat([data,fixed_dfs[i]], ignore_index = True)
data = data.dropna()
data['rrsult']=data['result'].apply(lambda x: 'buy' if x==1.0 else 'sell')
xx = pd.get_dummies(data["rrsult"])
data = pd.concat([data,xx],axis=1)
X = data.drop(["result","rrsult","buy","sell"],axis=1)
y = data["sell"]
y2 = data["buy"]
y = pd.concat([y,y2],axis=1)
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=11)
print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTest.shape)

(708, 16)
(708, 2)
(79, 16)
(79, 2)


In [86]:
data

,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,ten_days_ago,nine_days_ago,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result,rrsult,buy,sell
0,0.567130,0.753546,0.482063,0.000000,0.172682,0.125573,0.087662,0.059718,0.214019,0.315275,0.059089,0.087263,0.453781,0.750140,1.000000,0.709999,1.0,buy,1,0
1,0.729151,0.466456,0.000000,0.167091,0.121508,0.084824,0.057784,0.207090,0.305068,0.057176,0.084438,0.439089,0.725854,0.967625,0.687013,1.000000,0.0,sell,0,1
2,0.466456,0.000000,0.167091,0.121508,0.084824,0.057784,0.207090,0.305068,0.057176,0.084438,0.439089,0.725854,0.967625,0.687013,1.000000,0.904792,1.0,buy,1,0
3,0.000000,0.159404,0.115917,0.080921,0.055126,0.197562,0.291032,0.054545,0.080553,0.418888,0.692459,0.923107,0.655406,0.953993,0.863165,1.000000,1.0,buy,1,0
4,0.080334,0.047018,0.020207,0.000445,0.109568,0.181177,0.000000,0.019925,0.279129,0.488717,0.665420,0.460329,0.689082,0.619497,0.724329,1.000000,1.0,buy,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,0.000000,0.057958,0.068729,0.154335,0.095332,0.350094,0.439853,1.000000,0.525175,0.363735,0.420792,0.567469,0.454751,0.527685,0.414183,0.384630,0.0,sell,0,1
783,0.000000,0.011433,0.102306,0.039673,0.310109,0.405390,1.000000,0.495962,0.324590,0.385156,0.540858,0.421205,0.498627,0.378142,0.346770,0.071187,1.0,buy,1,0
784,0.000000,0.091924,0.028566,0.302130,0.398513,1.000000,0.490132,0.316778,0.378045,0.535547,0.414511,0.492828,0.370950,0.339215,0.060444,0.138533,1.0,buy,1,0
785,0.065221,0.000000,0.281608,0.380826,1.000000,0.475139,0.296687,0.359756,0.521889,0.397294,0.477914,0.352451,0.319784,0.032815,0.113200,0.262972,1.0,buy,1,0


In [99]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
#logisticRegr = LogisticRegression()
#logisticRegr.fit(xTrain, yTrain)
#print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
#nc = NearestCentroid()
#nc.fit(xTrain,yTrain)
#print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
#gnb = GaussianNB()
#gnb.fit(xTrain, yTrain)
#print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
#gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
#   max_depth=1, random_state=0).fit(xTrain, yTrain)
#gb.fit(xTrain,yTrain)
#print(f"Gradient Boost: {gb.score(xTest, yTest)}")
#svm = SVC()
#svm.fit(xTrain,yTrain)
#print(f"SVM: {svm.score(xTest,yTest)}")

Random Forest :  0.6582278481012658
K-Neighbors: 0.45569620253164556
Decision Tree: 0.5949367088607594


In [100]:
model = Sequential()
model.add(Dense(202, activation='relu', kernel_initializer='he_normal', input_shape=(16,)))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 202)               3434      
                                                                 
 dense_18 (Dense)            (None, 2)                 406       
                                                                 
Total params: 3,840
Trainable params: 3,840
Non-trainable params: 0
_________________________________________________________________


In [101]:
model.fit(xTrain,yTrain,epochs=75,batch_size=2,validation_data=(xTest,yTest))

Epoch 1/75
354/354 [==============================] - 1s 2ms/step - loss: 0.7028 - accuracy: 0.5297 - val_loss: 0.6726 - val_accuracy: 0.5570
Epoch 2/75
354/354 [==============================] - 1s 2ms/step - loss: 0.6438 - accuracy: 0.6469 - val_loss: 0.6980 - val_accuracy: 0.5949
Epoch 3/75
354/354 [==============================] - 1s 2ms/step - loss: 0.6212 - accuracy: 0.6681 - val_loss: 0.6961 - val_accuracy: 0.5696
Epoch 4/75
354/354 [==============================] - 1s 2ms/step - loss: 0.6028 - accuracy: 0.6525 - val_loss: 0.6994 - val_accuracy: 0.6456
Epoch 5/75
354/354 [==============================] - 1s 2ms/step - loss: 0.5780 - accuracy: 0.6893 - val_loss: 0.7022 - val_accuracy: 0.5443
Epoch 6/75
354/354 [==============================] - 1s 2ms/step - loss: 0.5735 - accuracy: 0.7090 - val_loss: 0.7172 - val_accuracy: 0.5190
Epoch 7/75
354/354 [==============================] - 1s 2ms/step - loss: 0.5594 - accuracy: 0.7020 - val_loss: 0.6731 - val_accuracy: 0.5823
Epoch 